In [1]:
# ! pip install torch==2.1.0  torchvision==0.16.0 torchtext==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu121
# #! pip install pyg_lib torch_scatter torch_sparse torch_cluster -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html # torch_spline_conv
# ! pip install torch_geometric
# ! pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cu121.html
# #! pip install torch_sparse -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
# #! pip install torch_scatter -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
# #! pip install pyg_lib -f https://data.pyg.org/whl/torch-2.1.0+${CUDA}.html
# ! pip install sentence-transformers
# ! pip install torcheval
# ! pip install matplotlib
# ! pip install pandas
# ! pip install tensorboard

In [2]:

# from graph_sampler import get_datasets, equal_edgeweight_hgt_sampler, get_minibatch_count, add_reverse_edge_original_attributes_and_label_inplace, get_hgt_linkloader, get_single_minibatch_count, sampler_for_init
import pickle 
import torch
import os 
from torch_geometric.data import HeteroData
def load_heterograph_dataset(folder):
    dataset = HeteroData()
    for name in os.listdir(folder):
        print('load', name)
        path = os.path.join(folder, name)
        if os.path.isfile(path):
            data = torch.load(path)
            for key, value in data['attributes'].items():
                dataset[data['name']][key] = value
    return dataset

data = load_heterograph_dataset('data_arxiv_paper')#pickle.load(open('/kaggle/input/arxiv-paper-graph/train_data_arxiv_paper.pkl', 'rb'))
#val_data = pickle.load(open('/kaggle/input/arxiv-paper-graph/val_data_arxiv_paper.pkl', 'rb'))
#test_data = pickle.load(open(filename, 'rb'))

#train_data, val_data, test_data = get_datasets(get_edge_attr=False, filter_top_k=True, top_k=15)

c:\Users\I549512\AppData\Local\miniconda3\envs\gnnpapersearch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


load author.pt
load author_rev_written_by_paper.pt
load category.pt
load category_rev_has_category_paper.pt
load journal.pt
load journal_rev_in_journal_paper.pt
load paper.pt
load paper_has_category_category.pt
load paper_has_titleword_word.pt
load paper_has_word_word.pt
load paper_in_journal_journal.pt
load paper_written_by_author.pt
load word.pt
load word_co_occurs_with_word.pt
load word_rev_co_occurs_with_word.pt
load word_rev_has_titleword_paper.pt
load word_rev_has_word_paper.pt


In [3]:
from torch_geometric.nn.kge import TransE

import math

import torch
import torch.nn.functional as F
from torch import Tensor

from torch_geometric.nn.kge import KGEModel

# adapted and taken from https://github.com/pyg-team/pytorch_geometric/blob/master/torch_geometric/nn/kge/transe.py

class TransE(KGEModel):
    r"""The TransE model from the `"Translating Embeddings for Modeling
    Multi-Relational Data" <https://proceedings.neurips.cc/paper/2013/file/
    1cecc7a77928ca8133fa24680a88d2f9-Paper.pdf>`_ paper.

    :class:`TransE` models relations as a translation from head to tail
    entities such that

    .. math::
        \mathbf{e}_h + \mathbf{e}_r \approx \mathbf{e}_t,

    resulting in the scoring function:

    .. math::
        d(h, r, t) = - {\| \mathbf{e}_h + \mathbf{e}_r - \mathbf{e}_t \|}_p

    .. note::

        For an example of using the :class:`TransE` model, see
        `examples/kge_fb15k_237.py
        <https://github.com/pyg-team/pytorch_geometric/blob/master/examples/
        kge_fb15k_237.py>`_.

    Args:
        num_nodes (int): The number of nodes/entities in the graph.
        num_relations (int): The number of relations in the graph.
        hidden_channels (int): The hidden embedding size.
        margin (int, optional): The margin of the ranking loss.
            (default: :obj:`1.0`)
        p_norm (int, optional): The order embedding and distance normalization.
            (default: :obj:`1.0`)
        sparse (bool, optional): If set to :obj:`True`, gradients w.r.t. to the
            embedding matrices will be sparse. (default: :obj:`False`)
    """
    def __init__(
        self,
        num_nodes: int,
        num_relations: int,
        hidden_channels: int,
        margin: float = 1.0,
        p_norm: float = 1.0,
        sparse: bool = False,
    ):
        super().__init__(num_nodes, num_relations, hidden_channels, sparse)

        self.p_norm = p_norm
        self.margin = margin

        self.reset_parameters()

    def reset_parameters(self):
        bound = 6. / math.sqrt(self.hidden_channels)
        torch.nn.init.uniform_(self.node_emb.weight, -bound, bound)
        torch.nn.init.uniform_(self.rel_emb.weight, -bound, bound)
        F.normalize(self.rel_emb.weight.data, p=self.p_norm, dim=-1,
                    out=self.rel_emb.weight.data)

    def forward(
        self,
        head_embeddings: Tensor,
        rel_type,
        tail_embeddings: Tensor,
    ) -> Tensor:
        #head = self.node_emb(head_index)
        rel = self.rel_emb(rel_type)  # Amos: only learn the relation embeddings, others are learned with GNN
        #tail = self.node_emb(tail_index)

        head = F.normalize(head_embeddings, p=self.p_norm, dim=-1)
        tail = F.normalize(tail_embeddings, p=self.p_norm, dim=-1)
        # Calculate *negative* TransE norm:
        negative_norm = -((head + rel) - tail).norm(p=self.p_norm, dim=-1)
        return negative_norm

    
    def get_embedding(self,
                      embedding,
                    #   rel_type,
                        # have_head_or_tail
                      ):
        # rel = self.rel_emb(rel_type)
        embedding = F.normalize(embedding, p=self.p_norm, dim=-1)
        # if have_head_or_tail == 'head':
        #     return embedding + rel
        # else:
        #     return embedding - rel
        return embedding
    
    def get_relationship_embedding(self,rel_type):
        return self.rel_emb(rel_type)
    
    
    def loss(
        self,
        head_embeddings: Tensor,
        rel_type: Tensor,
        tail_embeddings: Tensor,
        labels: Tensor, # labels 0 or 1
    ) -> Tensor:
        pos_mask = labels == 1
        neg_mask = labels == 0
        
        pos_score = self(head_embeddings[pos_mask], rel_type, tail_embeddings[pos_mask])
        neg_score = self(head_embeddings[neg_mask], rel_type, tail_embeddings[neg_mask])
        loss = F.margin_ranking_loss(
            pos_score,
            neg_score,
            target=torch.ones_like(pos_score), # 1 for similarity, -1 for dissimilarity
            margin=self.margin,
        )
        return loss

In [4]:
from torch_geometric.nn import HGTConv, Linear
import torch 

class HGT(torch.nn.Module):
    def __init__(self, hidden_channels, out_channels, num_heads, num_layers, node_types, data_metadata):
        super().__init__()

        self.lin_dict = torch.nn.ModuleDict()
        for node_type in node_types:
            self.lin_dict[node_type] = Linear(-1, hidden_channels)

        self.convs = torch.nn.ModuleList()
        for _ in range(num_layers):
            conv = HGTConv(hidden_channels, hidden_channels, data_metadata,
                           num_heads, group='sum')
            self.convs.append(conv)

        self.lin = Linear(hidden_channels, out_channels)

    def forward(self, x_dict, edge_index_dict):
        x_dict = {
            node_type: self.lin_dict[node_type](x).relu_()
            for node_type, x in x_dict.items()
        }

        for conv in self.convs:
            x_dict = conv(x_dict, edge_index_dict)

        return x_dict
    
# if __name__ == '__main__':
    # model = HGT(hidden_channels=64, out_channels=4, num_heads=2, num_layers=1, node_types=data.node_types, data_metadata=data.metadata())

In [5]:
import torch
# from models.TransE import TransE
# from models.DistMult import DistMult
# from models.HGT import HGT
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Model(torch.nn.Module):
    def __init__(self, gnn : torch.nn.Module, head :  torch.nn.Module, node_types, edge_types, ggn_output_dim, pnorm=1):
        super().__init__()
        # edge_type onehot lookup table with keys
        # node_type onehot lookup table with keys
        self.node_type_embedding = torch.nn.Embedding(len(node_types), ggn_output_dim) # hidden channels should be the output dim of gnn

        self.edge_types = edge_types
        for edge_type in edge_types:
            if edge_type[1].startswith('rev_'):
                self.edge_types.remove(edge_type)

        # create edge to int mapping
        self.edgeindex_lookup = {edge_type:torch.tensor(i)  for i, edge_type in enumerate(edge_types)}

        # hidden channels should be the output dim of gnn
        if head=='TransE':
            self.head = TransE(len(node_types), len(edge_types) , ggn_output_dim, p_norm= pnorm, margin=0.5)  # KGE head with loss function
        else:
            raise NotImplementedError

        self.gnn = gnn



    def forward(self, hetero_data1, target_edge_type, edge_label_index, edge_label, hetero_data2=None, get_head_fn='loss'):

        if hetero_data2 is not None:
            assert target_edge_type[0] != target_edge_type[2], 'when passing two data objects, the edge type has to contain two different node types'
            head_embeddings = self.gnn(hetero_data1.x_dict, hetero_data1.edge_index_dict)[target_edge_type[0]][edge_label_index[0,:]]
            tail_embeddings = self.gnn(hetero_data2.x_dict, hetero_data2.edge_index_dict)[target_edge_type[2]][edge_label_index[1,:]]
        else:
            assert target_edge_type[0] == target_edge_type[2], 'when passing one data object, the edge type has to contain the same node types'


            embeddings = self.gnn(hetero_data1.x_dict, hetero_data1.edge_index_dict)
            head_embeddings = embeddings[target_edge_type[0]][edge_label_index[0,:]]
            tail_embeddings = embeddings[target_edge_type[2]][edge_label_index[1,:]]
        

        edgeindex = self.edgeindex_lookup[target_edge_type]
        if get_head_fn=='loss':
            loss = self.head.loss(head_embeddings, edgeindex.to(device), tail_embeddings, edge_label)
            return loss
        elif get_head_fn=='forward':
            return self.head.forward(head_embeddings, edgeindex.to(device), tail_embeddings)
        
    def get_embeddings(self, hetero_data, node_type, first_n_nodes):
        embeddings = self.gnn(hetero_data.x_dict, hetero_data.edge_index_dict)[node_type][:first_n_nodes,:]
        embeddings = self.head.get_embedding(embeddings)
        return embeddings

    def get_relationship_embedding(self, edge_type):
        return self.head.get_relationship_embedding(model.edgeindex_lookup[edge_type].cuda())

metadata = data.metadata()
# add selfloops
for node_type in data.node_types:
    metadata[1].append((node_type, 'self_loop', node_type))

out_channels = 256
hidden_channels = 256
num_heads = 8
num_layers = 2
pnorm = 2
head = 'TransE'
gnn = HGT(hidden_channels=out_channels, out_channels=out_channels, num_heads=num_heads, num_layers=num_layers, node_types=data.node_types, data_metadata=metadata)

model = Model(gnn, head=head, node_types=metadata[0], edge_types=metadata[1], ggn_output_dim=out_channels, pnorm=pnorm)
#torch_geometric.compile(model, dynamic=True)
model.to(device)
model.eval()

Model(
  (node_type_embedding): Embedding(5, 256)
  (head): TransE(5, num_relations=13, hidden_channels=256)
  (gnn): HGT(
    (lin_dict): ModuleDict(
      (author): Linear(-1, 256, bias=True)
      (category): Linear(-1, 256, bias=True)
      (journal): Linear(-1, 256, bias=True)
      (paper): Linear(-1, 256, bias=True)
      (word): Linear(-1, 256, bias=True)
    )
    (convs): ModuleList(
      (0-1): 2 x HGTConv(-1, 256, heads=8)
    )
    (lin): Linear(256, 256, bias=True)
  )
)

In [6]:
# COMMAND ----------
# init model
from tqdm.auto import tqdm
from datetime import datetime
batch_size = 32
num_node_types = len(data.node_types)
print('num_node_types', num_node_types)
one_hop_neighbors = (20 * batch_size)//num_node_types # per relationship type
two_hop_neighbors = (20 * 8 * batch_size)//num_node_types # per relationship type
three_hop_neighbors = (20 * 8 * 3 * batch_size)//num_node_types # per relationship type
num_neighbors = [one_hop_neighbors, two_hop_neighbors] # three_hop_neighbors

from torch_geometric.loader import HGTLoader

num_node_types 5


In [7]:
def get_embeddings(node_type):
    
    loader = HGTLoader(
            data,
            # Sample 512 nodes per type and per iteration for 4 iterations
            num_samples=num_neighbors,
            batch_size=64, #96 or 32 nodes
            input_nodes=node_type,
            num_workers=4,
            pin_memory=True,
            prefetch_factor=None,
        )
    

In [8]:
# loader = HGTLoader(
#             data,
#             # Sample 512 nodes per type and per iteration for 4 iterations
#             num_samples=num_neighbors,
#             batch_size=64, #96 or 32 nodes
#             input_nodes=node_type,
#             num_workers=4,
#             pin_memory=True,
#             prefetch_factor=None,
#         )
# minibatch = next(iter(loader))

In [11]:

# init
model.eval()
for node_type in data.node_types:
    print(node_type)
    
    loader = HGTLoader(
            data,
            # Sample 512 nodes per type and per iteration for 4 iterations
            num_samples=num_neighbors,
            batch_size=64, #96 or 32 nodes
            input_nodes=node_type,
            num_workers=0,
            pin_memory=True,
            prefetch_factor=None,
        )
    minibatch = next(iter(loader))
    
    model.get_embeddings(minibatch.cuda(), node_type, first_n_nodes=64)

author
category
journal
paper
word


In [16]:
minibatch

HeteroData(
  author={
    x=[434, 385],
    n_id=[434],
  },
  category={
    x=[92, 385],
    n_id=[92],
  },
  journal={
    x=[48, 385],
    n_id=[48],
  },
  paper={
    x=[1152, 400],
    n_id=[1152],
  },
  word={
    x=[1216, 386],
    n_id=[1216],
    input_id=[64],
    batch_size=64,
  },
  (author, rev_written_by, paper)={
    edge_index=[2, 411],
    e_id=[411],
  },
  (category, rev_has_category, paper)={
    edge_index=[2, 1648],
    e_id=[1648],
  },
  (journal, rev_in_journal, paper)={
    edge_index=[2, 729],
    e_id=[729],
  },
  (paper, has_category, category)={
    edge_index=[2, 2],
    e_id=[2],
  },
  (paper, has_titleword, word)={
    edge_index=[2, 209],
    e_id=[209],
  },
  (paper, has_word, word)={
    edge_index=[2, 366],
    e_id=[366],
  },
  (paper, in_journal, journal)={
    edge_index=[2, 47],
    e_id=[47],
  },
  (paper, written_by, author)={
    edge_index=[2, 384],
    e_id=[384],
  },
  (word, co_occurs_with, word)={
    edge_index=[2, 1558],
  

In [14]:
# load state dict
model_and_optimizer = torch.load('cache/runs/all_hgt_20240113_003259_mg05_norm2_lr0_0002_bs32_neigh_128_1024_h_TransE_hid_256_out_256_numh_8_numl_2/model_samplesseen144000.pt')
model.load_state_dict(model_and_optimizer['model_state_dict'])

<All keys matched successfully>

In [20]:
minibatch['author']

{'x': tensor([[-0.0187,  0.0200, -0.0051,  ...,  0.0610,  0.0057,  0.0390],
        [-0.0636,  0.0415, -0.0116,  ...,  0.0699,  0.0469,  0.0111],
        [-0.0832,  0.0122,  0.0189,  ..., -0.0008, -0.0708,  0.0419],
        ...,
        [-0.0215,  0.0772, -0.0655,  ...,  0.0012,  0.0276,  0.0010],
        [-0.0740,  0.1247,  0.0085,  ..., -0.0013, -0.0143,  0.0010],
        [ 0.0291,  0.0830, -0.0319,  ...,  0.0141,  0.0307,  0.0010]],
       device='cuda:0'), 'n_id': tensor([ 574757,  118875,    8489,       2,  279889, 1315707,    5907,  564852,
         878933,  214933, 1217613,       3,  880886, 1819963, 1766766,  532574,
         581592,  251337, 1533046,  537412,  449671,   21633,  546957,  133737,
         736075, 1421581,   49420,  517543, 1809497,   11259,  142743, 1819962,
         125280,  957385,   35208,  560719, 1819961,  937233,       0,   47225,
         888980,       1,  935556,  206517,  614296,  512782, 1235437, 1286483,
        1387542, 1235438,   61879,  614298,   7

In [31]:
model.eval()
with torch.no_grad():
    for node_type in data.node_types:
        batch_size = {
            'paper': 32,
            'author': 64,
            'word': 64,
            'journal': 64,
            'category': 64,
            }[node_type]
        
        print(node_type)
        
        
        loader = HGTLoader(
                data,
                # Sample 512 nodes per type and per iteration for 4 iterations
                num_samples=num_neighbors,
                batch_size=batch_size, #96 or 32 nodes
                input_nodes=node_type,
                num_workers=2,
                pin_memory=True,
                prefetch_factor=2,
                drop_last=False,
                shuffle=True
            )
        
        embeddings = []
        indices = []
        for minibatch in tqdm(loader):
            embds = model.get_embeddings(minibatch.cuda(), node_type, first_n_nodes=batch_size).cpu()
            n_ids = minibatch[node_type].n_id[:batch_size]
            embeddings.append(embds)
            indices.append(n_ids)
            # for i, n_id in enumerate(n_ids):
                # embeddings[n_id.item()] = embds[i,:]
        embeddings = torch.cat(embeddings)
        indices = torch.cat(indices)
        torch.save({'embeddings':embeddings, 'indices':indices}, f'embeddings_{node_type}.pt')
        
    for edge_type in data.edge_types:
        if 'rev_' in edge_type[1]:
            continue
        
        rel_embd = model.get_relationship_embedding(edge_type)
        torch.save({'embedding':edge_type}, f'relationship_embedding_{"_".join(edge_type)}.pt')

word


  0%|          | 1/38514 [00:00<8:34:16,  1.25it/s]

tensor([1929350, 1234238, 1052857,   24176, 1062789, 2218334, 1021327, 1553884,
         334198, 2387895, 1955452, 1524628, 1832836, 1791221, 2291320, 1234940,
        2285470,  983243, 2452261,  157912, 1051691,  896719, 1803973, 1336140,
        2041420, 1962366,  646210,  430938, 1788147,  917328,  365214, 2420617,
        2097264,  861585, 2289217, 1521202,  724826,  310666,  867373, 1779422,
        1892927,  976410, 1897790, 1511447,  490173,  232810, 1237916, 2207901,
         282487, 1517768, 1003755,  802259,  273517, 1990207, 2199369,  310526,
          24626, 2233566, 1794842, 1347469, 2222332, 1926640, 2334614, 1207183],
       device='cuda:0')


  0%|          | 2/38514 [00:01<8:01:42,  1.33it/s]

tensor([2352534, 1118889, 1833076, 1495366, 1707582, 2131318, 1826475, 1960888,
         418856, 2268695, 1513982,  904619, 1145331,  549565,  382744,  911450,
         668794,  434865, 1244411, 1580404, 2240430, 1827070, 1024289, 1366218,
        1526758,  361713, 1020940, 1400037, 1656216, 1044190,   68146, 2209638,
         132706,  399811, 1375511, 1229170,   23393,  570548, 1497944,  682022,
        2268862, 2282350,  756856,  704053, 2197343,  301648, 2082394, 1917085,
         562194, 2051955,  522113, 2422560, 1823937, 1846828,  256785, 2184119,
          22300, 1928596, 2089044, 1571096,  908107, 2254253,  324958, 1605999],
       device='cuda:0')


  0%|          | 3/38514 [00:02<7:12:58,  1.48it/s]

tensor([1643097, 1117584, 1734158, 2284907,  112186,  580848,  164502,  123386,
         405493,  682864, 1310955,  481352,  298911,  619920, 1280591,   31756,
        1145425, 1465802, 1762216, 2294221, 1934642, 1441156, 1038040, 1172363,
        1033701,  876439, 1328757, 1139169, 2428074, 2190089, 1106117, 1356273,
        1862284, 1108181,  259659, 1260329,  716359,   59948,  684694,  892477,
        2419862, 2154552, 1578410, 1067506,  712412, 1296457,  144309,  704041,
         105701,  824653, 1082605, 1706165, 1458115,  170907,  616559,  357668,
         825183, 1113146, 1334531,  461522, 1268622,  389055,  980914,  835528],
       device='cuda:0')


  0%|          | 3/38514 [00:02<8:51:47,  1.21it/s]


KeyboardInterrupt: 

In [ ]:
model.edgeindex_lookup